In [5]:
filepath_en = 'train.en'
filepath_hi = 'train.hi'

with open(filepath_en, encoding='utf-8') as file:
    lines_en = file.readlines()
print(len(lines_en))
print(lines_en[:5])

with open(filepath_hi, encoding='utf-8') as file:
    lines_hi = file.readlines()

print(len(lines_hi))
print(lines_hi[:5])

10125706
["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles\n", 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.\n', 'The value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.\n', 'Mithali To Anchor Indian Team Against Australia in ODIs\n', 'After the assent of the Honble President on 8thSeptember, 2016, the 101thConstitutional Amendment Act, 2016 came into existence\n']
10125706
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।\n', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है\n', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प

In [43]:
TOTAL_SENTENCES = 200000
english_sentences = lines_en[:TOTAL_SENTENCES]
hindi_sentences = lines_hi[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

print(english_sentences[:5])
print("================================")
print(hindi_sentences[:5])

["however, paes, who was partnering australia's paul hanley, could only go as far as the quarterfinals where they lost to bhupathi and knowles", 'whosoever desires the reward of the world, with allah is the reward of the world and of the everlasting life. allah is the hearer, the seer.', 'the value of insects in the biosphere is enormous because they outnumber all other living groups in measure of species richness.', 'mithali to anchor indian team against australia in odis', 'after the assent of the honble president on 8thseptember, 2016, the 101thconstitutional amendment act, 2016 came into existence']
['आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।', 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है', 'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले मे

In [44]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 258.0
97th percentile length English: 267.0


In [45]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\' ', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

hindi_vocabulary = [START_TOKEN, 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ',
    'क', 'ख', 'ग', 'घ', 'ङ','ड़', 'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श',
    'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'े', 'ै', 'ो', 'ौ',
    'ं', 'ः', 'ँ', '्', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', '|',
    ':', '<', '=', '>', '?', '@',
    '[', '\' ', ']', '^', '_', '`',
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9','\u200d','।', 'ृ',
    PADDING_TOKEN, END_TOKEN
]




In [46]:
max_sequence_length = 250

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")


Number of sentences: 200000
Number of valid sentences: 151260


In [47]:
# for i in 'शख्स':
  # print(i)


# hindi_sentences = [
#     'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
#     'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है',
#     'जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि प्रजातियों की समृद्धि के मामले में उनकी संख्या अन्य जीव समूहों से ज़्यादा है।',
#     'आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को',
#     '8 सितम्‍बर, 2016 को माननीय राष्ट्रपति की स्‍वीकृति मिलने के बाद 101वां संविधान संशोधन अधिनियम, 2016 अस्तित्व में आया'
# ]

# Define the provided vocabulary set
hindi_vocabulary_set = set(hindi_vocabulary)

# Combine all sentences into one string
combined_sentences = ''.join(hindi_sentences)

# Find all unique characters in the combined sentences
unique_chars_in_sentences = set(combined_sentences)

# Find characters in the sentences that are not in the vocabulary
missing_chars = unique_chars_in_sentences - hindi_vocabulary_set

print("Characters in the sentences not in the vocabulary:", missing_chars)


Characters in the sentences not in the vocabulary: {'現', 'ಮ', 'j', 'z', 'а', 'ᐅ', 'ז', 'س', '\ue208', '郑', '§', 'K', 'P', 'ː', '্', 'ग़', 'æ', 'រ', 'þ', '්', 'ו', '車', 'ਤ', 'پ', 'ऑ', 'ਰ', '\uf0b7', 'ท', 'ோ', 'য', '£', 'я', 'ಯ', 'ч', 'ֶ', '\uf642', '□', 'a', 'о', 'ස', 'ל', 'ä', 'ţ', 'ę', 'ট', '\u2061', 'ế', 'ᠠ', 'ऎ', 'A', '“', 'ب', 'ុ', 'Ć', 'פ', 'ย', '⚡', '₹', 'ö', '洞', 'Ż', '魯', 'ַ', 'ᓗ', 'ב', 'ل', 'μ', '⋅', 'ග', 'ঙ', 'ě', 'ִ', '林', 'ស', '\xad', 'ਸ', 'ನ', 'J', 'е', 'É', 'ො', 'ロ', 'ٓ', '庭', '⁻', 'H', 'জ', '̈', '̯', '西', 'ミ', '°', '\u200e', 'Ō', 'í', 'O', 'ï', 'ක', 'λ', 'h', 'ה', '⊆', 'б', 'E', 'ּ', '域', '○', 'త', '©', 'ऽ', '్', 'ˈ', 'D', '’', 'ং', 'ಕ', 'ு', 'n', 'স', 'ඩ', '휴', 'ô', 'К', 'த', 'ó', 'ਵ', 'آ', 'ο', 'ॊ', 'ಿ', 'ō', '\ufeff', 'ऌ', '의', '한', '್', 'ল', '℃', 'χ', 'パ', 'Â', '»', 'ऍ', 'চ', 'ی', 'Ò', 'ত', 'ម', 'β', 'ॆ', 'ع', 'C', 'ہ', '॥', '¸', 'ਦ', '្', '日', 'מ', 'ᠨ', '\uf0d8', 'â', '›', 'إ', 'л', '班', 'ය', 'க', 'i', 'X', '夕', '에', '\u200f', 'м', 'Г', '自', 'ದ', 'k', 'ه', 'ρ', 'Ę', 